In [0]:
import pfs.drp.stella.findAndTraceAperturesTask as fataTask
import pfs.drp.stella.createFlatFiberTraceProfileTask as cfftpTask
import pfs.drp.stella.extractSpectraTask as esTask
import pfs.drp.stella as drpStella
import numpy as np
import matplotlib.pyplot as plt
import pdb
import lsst.daf.persistence as dafPersist
import lsst.afw.cameraGeom.utils as cgUtils
import lsst.afw.display.ds9 as ds9
from astropy.io import fits as pyfits
%matplotlib inline

In [0]:
#%run /Users/azuri/stella-git/drp_stella/python/pfs/drp/stella/demo.py '/Volumes/My Passport/data/spectra/pfs/temp/PFS' --calib='/Volumes/My Passport/data/spectra/pfs/temp/PFS/CALIB' --id site='F' category='A' det=2 filter='PFS-R' -c isr.doBias=False isr.doDark=False isr.doFlat=False isr.doFringe=False isr.removePcCards=False isr.doDefect=False isr.qa.doThumbnailOss=False isr.doLinearize=False isr.qa.doThumbnailFlattened=False isr.doGuider=False isr.doWriteVignettePolygon=False --doraise 

In [0]:
butler = dafPersist.Butler("/Volumes/My Passport/data/spectra/pfs/PFS")

In [0]:
flat = butler.get('postISRCCD', visit=5, filter='PFS-R', spectrograph=2, site='F', category='A', immediate=True)
print 'type(flat) = ',type(flat)


In [0]:
frame=1
ds9.mtv(flat, frame=frame, title='flat')


In [0]:
myFindTask = fataTask.FindAndTraceAperturesTask()
fts = myFindTask.run(flat)
print fts.size()
#for i in range(fts.size()):
#    print 'FiberTrace ',i
    print fts.getFiberTrace(i).getXCenters().shape[0]
    print fts.getFiberTrace(i).getHeight()
    print fts.getFiberTrace(i).getFiberTraceFunction().yLow
    print fts.getFiberTrace(i).getFiberTraceFunction().yHigh
    length = fts.getFiberTrace(i).getFiberTraceFunction().yHigh - fts.getFiberTrace(i).getFiberTraceFunction().yLow + 1
    print length



In [0]:
# TEST CreateFlatFiberTraceProfileTask
cfftpTask = reload(cfftpTask)

print fts.getFiberTrace(fts.size()-1).getXCenters().shape[0]
print fts.getFiberTrace(fts.size()-1).getHeight()
print fts.getFiberTrace(fts.size()-1).getFiberTraceFunction().yLow
print fts.getFiberTrace(fts.size()-1).getFiberTraceFunction().yHigh
length = fts.getFiberTrace(fts.size()-1).getFiberTraceFunction().yHigh -  fts.getFiberTrace(fts.size()-1).getFiberTraceFunction().yLow + 1
print length

myProfileTask = cfftpTask.CreateFlatFiberTraceProfileTask()

"""calculate flat profiles"""
myProfileTask.run(fts)

print "fts.getFiberTrace(0).getProfile().getArray() = "
print fts.getFiberTrace(0).getProfile().getArray()[0,:]
print " "
print "fts.getFiberTrace(1).getProfile().getArray()"
print fts.getFiberTrace(1).getProfile().getArray()[0,:]
print " "
print "fts.getFiberTrace(2).getProfile().getArray()"
print fts.getFiberTrace(2).getProfile().getArray()[0,:]
print 'fts.getFiberTrace(fts.size()-1) = ',fts.getFiberTrace(fts.size()-1)
print 'fts.getFiberTrace(fts.size()-1).getXCenters() = ',fts.getFiberTrace(fts.size()-1).getXCenters()
print len(fts.getFiberTrace(fts.size()-1).getXCenters())
print fts.getFiberTrace(fts.size()-1).getHeight()
print fts.getFiberTrace(fts.size()-1).getFiberTraceFunction().yLow
print fts.getFiberTrace(fts.size()-1).getFiberTraceFunction().yHigh
length = fts.getFiberTrace(fts.size()-1).getFiberTraceFunction().yHigh -  fts.getFiberTrace(fts.size()-1).getFiberTraceFunction().yLow + 1
print length

"""extract flats from profiles"""
spectrumSet = fts.extractAllTracesFromProfile()
if False:
    for i in range(spectrumSet.size()):
        plt.plot(spectrumSet.getSpectrum(i).getSpectrum(),'-+')
        plt.xlim(1000,1500)



In [0]:
arc = butler.get('postISRCCD', visit=4, filter='PFS-R', spectrograph=2, site='F', category='A', immediate=True)


In [0]:
frame=2
ds9.mtv(arc, frame=frame, title='arc')


In [0]:
myExtractTask = esTask.ExtractSpectraTask()

"""Extract all apertures"""
iAperturesToExtract = [-1]#np.arange(0,2,1,int)
spectrumSetFromProfile = myExtractTask.run(arc, fts, iAperturesToExtract)
print 'type(spectrumSetFromProfile.getSpectrum(0).getSpectrum()[0]) = ',type(spectrumSetFromProfile.getSpectrum(0).getSpectrum()[0])

if True:
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    for i in range(spectrumSetFromProfile.size()):
        ax.plot(spectrumSetFromProfile.getSpectrum(i).getSpectrum(),'-+')
        plt.xlim(1450,1600)
        plt.ylim(0,8000)
    plt.show()
    plt.close(fig)
    fig.clf()

In [0]:
refSpec = spectrumSetFromProfile.getSpectrum(int(spectrumSetFromProfile.size() / 2))
ref = refSpec.getSpectrum()
print ref
#hdu = pyfits.PrimaryHDU(ref)
#hdu.writeto('/Users/azuri/stella-git/obs_Subaru/pfs/arcSpectra/refSpec_CdHgKrNeXe_red.fits')

In [0]:
fileName = '/Users/azuri/stella-git/obs_Subaru/pfs/lineLists/CdHgKrNeXe_wlen_strength.txt'
dType = 'float32'
%run readFile.ipynb
#print dataArr
tempArr = np.arange(3500., 13000., 1.)
print tempArr.shape
plotArr = np.ndarray(shape=(tempArr.shape[0] + dataArr.shape[0], 2))
print plotArr.shape
iWLen = 0
iX = 0
iTemp = 0
for x in tempArr:
    if iWLen < dataArr.shape[0]:
        if x > dataArr[iWLen][0]:
            plotArr[iX][:] = dataArr[iWLen][:]
#            print 'plotArr[',iX,'][:] set to ',plotArr[iX][:]
            iWLen = iWLen + 1
#            print 'iWLen set to ',iWLen
            iX = iX + 1
#            print 'iX set to ',iX
    plotArr[iX][0] = tempArr[iTemp]
    plotArr[iX][1] = 0.
#    print 'plotArr[',iX,'][:] set to ',plotArr[iX][:]
    iTemp = iTemp + 1
#    print 'iTemp set to ',iTemp
    iX = iX + 1
#    print 'iX set to ',iX
#print plotArr

In [0]:
xRange = [7500.,9500.]
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(plotArr[:,0],plotArr[:,1],'b+-')
plt.xlim(xRange[0],xRange[1])
#plt.ylim(0,1000)
plt.show()
plt.close(fig)

#for (x,y) in plotArr[:][:]:
#    if x > xRange[0] and x < xRange[1]:
#        print x,y


In [0]:
# create line list fits file
fileName = '/Users/azuri/stella-git/obs_subaru/pfs/lineLists/CdHgKrNeXe_use.txt'
""" count lines """
nLines = 0
nDataLines = 0
with open(fileName, 'r') as inFile:
    while inFile.readline() != '':
        nLines = nLines + 1
#            print 'nLines = ',nLines
inFile.closed
nDataLines = nLines

""" read file to dataArr """
colWLen = np.ndarray(shape=(nDataLines), dtype='float')
colPix = np.ndarray(shape=(nDataLines), dtype='float')
colElement = np.empty(nDataLines, dtype='|S2')
#colStrength = np.empty(nDataLines, dtype='int')
colComment = np.empty(nDataLines, dtype='|S128')
iDataLine = 0
with open(fileName, 'r') as inFile:
    for iLine in np.arange(0,nLines,1):
        line = inFile.readline()
        print 'iLine = ',iLine,': line = <',line,'>'
        if len(line) > 0:
            if line[0] != '#':
                words = line.split()
                colWLen[iDataLine] = float(words[0])
                colPix[iDataLine] = float(words[1])
                print 'iLine = ',iLine,': colWLen[',iDataLine,'] = ',colWLen[iDataLine]
                print 'iLine = ',iLine,': colPix[',iDataLine,'] = ',colPix[iDataLine]
                colElement[iDataLine] = words[2]
                colComment[iDataLine] = ''
                print 'len(words) = ',len(words)
                for iWord in np.arange(3,len(words),1):
                    print 'colComment[',iDataLine,'] = ',colComment[iDataLine],', words[',iWord,'] = ',words[iWord]
                    colComment[iDataLine] = colComment[iDataLine] + words[iWord]
                    if iWord < len(words) - 1:
                        colComment[iDataLine] = colComment[iDataLine] + ' '
                    print 'colComment[',iDataLine,'] = <',colComment[iDataLine],'>'

                iDataLine = iDataLine + 1
inFile.closed
print 'colWLen = ',colWLen
print 'colPix = ',colPix
print 'colElement = ',colElement
print 'colComment = ',colComment

col1 = pyfits.Column(name='wavelength', format='E', array=colWLen)
col2 = pyfits.Column(name='pixel', format='E', array=colPix)
col3 = pyfits.Column(name='element', format='2A', array=colElement)
col4 = pyfits.Column(name='comment', format='128A', array=colComment)
cols = pyfits.ColDefs([col1, col2, col3, col4])
tbhdu = pyfits.BinTableHDU.from_columns(cols)
#tbhdu.writeto('/Users/azuri/stella-git/obs_subaru/pfs/lineLists/CdHgKrNeXe_red.fits')

In [0]:
hdulist = pyfits.open('/Users/azuri/stella-git/obs_subaru/pfs/lineLists/CdHgKrNeXe_red.fits')
tbdata = hdulist[1].data
print tbdata[0]
print tbdata[0][:]
print tbdata[:][0]

print len(tbdata)
lineList = np.ndarray(shape=(len(tbdata),2), dtype='float64')
print 'type(lineList) = ',type(lineList),', ',type(lineList[0,0])

lineList[:,0] = tbdata.field(0)
lineList[:,1] = tbdata.field(1)

print lineList.shape

In [0]:
dispCorControl = drpStella.DispCorControl()
print 'type(lineList) = ',type(lineList),', ',type(lineList[0,0])
refSpec.identifyD(lineList, dispCorControl)
print refSpec.getDispCoeffs()
print refSpec.getDispRms()
print refSpec.getWavelength()
print 'ref = ',ref.shape,': ',ref
col1 = pyfits.Column(name='flux', format='E', array=ref)
cols = pyfits.ColDefs([col1])
tbhdu = pyfits.BinTableHDU.from_columns(cols)
refSpecName = '/Users/azuri/stella-git/obs_subaru/pfs/lineLists/refCdHgKrNeXe_red.fits'
#tbhdu.writeto(refSpecName)

In [0]:
refSpecSpec = refSpec.getSpectrum()
#print 'type(specSpec) = ',type(specSpec),': ',type(specSpec[0])
print 'type(refSpecSpec) = ',type(refSpecSpec),': ',type(refSpecSpec[0])
print 'refSpecSpec.shape = ',refSpecSpec.shape
print 'type(lineList) = ',type(lineList),': ',type(lineList[0][0])

hdulist = pyfits.open(refSpecName)
tbdata = hdulist[1].data
refSpecArr = np.ndarray(shape=(len(tbdata)), dtype='float32')
refSpecArr[:] = tbdata.field(0)
print 'refSpecArr.shape = ',refSpecArr.shape

dispCorControl = drpStella.DispCorControl()
dispCorControl.radiusXCor = 50
dispCorControl.lengthPieces = 500
dispCorControl.nCalcs = 15
dispCorControl.stretchMinLength = 460
dispCorControl.stretchMaxLength = 540
dispCorControl.nStretches = 80

for i in range(spectrumSetFromProfile.size()):
    spec = spectrumSetFromProfile.getSpectrum(i)
    specSpec = spec.getSpectrum()
    result = drpStella.stretchAndCrossCorrelateSpecFD(specSpec,refSpecArr, lineList, dispCorControl)
    print result.lineList
    spec.identifyD(result.lineList, dispCorControl)
    print spec.getDispCoeffs()
    print spec.getDispRms()
    print spec.getWavelength()

In [0]:
DA1_ChiSquare = [2.29379e+07, 2.24558e+07, 2.20587e+07, 2.18063e+07, 2.17079e+07, 2.18065e+07, 2.21231e+07, 2.2699e+07, 2.40911e+07, 2.57779e+07, 2.63033e+07, 2.5984e+07, 2.58379e+07, 2.59771e+07, 2.61889e+07, 2.63426e+07, 2.64579e+07, 2.65831e+07, 2.67063e+07, 2.67886e+07, 2.6812e+07, 2.67894e+07, 2.6756e+07, 2.67363e+07, 2.66823e+07, 2.63613e+07, 2.53419e+07, 2.34224e+07, 2.12032e+07, 1.9763e+07, 1.93547e+07, 1.88487e+07, 1.73226e+07, 1.55852e+07, 1.52331e+07, 1.61924e+07, 1.64161e+07, 1.45775e+07, 1.21366e+07, 1.19469e+07, 1.49942e+07, 1.94745e+07, 2.30253e+07, 2.49164e+07, 2.56663e+07, 2.54008e+07, 2.40057e+07, 2.19789e+07, 2.04794e+07, 2.05845e+07, 2.22099e+07, 2.4291e+07, 2.56915e+07, 2.62494e+07, 2.64019e+07, 2.65161e+07, 2.66858e+07, 2.68723e+07, 2.70232e+07, 2.7089e+07, 2.69982e+07, 2.66649e+07, 2.60621e+07, 2.53356e+07, 2.47783e+07, 2.45044e+07, 2.42151e+07, 2.36003e+07, 2.29277e+07, 2.28571e+07, 2.35921e+07, 2.45958e+07, 2.52446e+07, 2.54563e+07, 2.507e+07, 2.38031e+07, 2.27216e+07, 2.27002e+07, 2.26453e+07, 2.18068e+07, 2.03837e+07, 1.91892e+07, 1.90827e+07, 2.01818e+07, 2.17574e+07, 2.29837e+07, 2.35624e+07, 2.36276e+07, 2.33596e+07, 2.29005e+07, 2.2489e+07, 2.24073e+07, 2.27413e+07, 2.32744e+07, 2.36654e+07, 2.36963e+07, 2.33635e+07, 2.27883e+07, 2.21268e+07, 2.15901e+07, 2.14439e+07]
plt.plot(DA1_ChiSquare)
#plt.xlim(37,40)

In [0]:
colormap = plt.cm.gist_ncar
plt.gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, result.specPieces.shape[2]/4)])

for i in range(result.specPieces.shape[2]):
    specPlot = result.specPieces[:,:,i]
    print 'specPlot.shape = ',specPlot.shape
    print 'specPlot = ',specPlot
    plt.plot(specPlot[:,0], specPlot[:,1])

plt.xlim(3500,4000)
plt.ylim(0,10000)